In [ ]:
# We will discuss about how to share models and tokenizers to HuggingFace
# We will discuss about how useful it will be like GitHub for your upcoming DL projects

In [ ]:
# we have already discussed about making pipeline() to run inference on loaded model and Auto* class to import model & tokenizer in various use-cases

3 ways to go about creating new model repositories:

- Using the push_to_hub API
- Using the huggingface_hub Python library
- Using the web interface
<br>

Steps:

- generate an authentication token so that the huggingface_hub API(set permissions of read, write, inference carefully) knows who you are and what namespaces you have write access to
- for terminal:
```bash
huggingface-cli login
```
- for notebook
```bash
from huggingface_hub import notebook_login
notebook_login()
```

You will be prompted for username and password.

In [1]:
!pip install huggingface_hub

In [2]:
from huggingface_hub import whoami , notebook_login

notebook_login()

user = whoami()
print(user)

HTTPError: Invalid user token. If you didn't pass a user token, make sure you are properly logged in by executing `huggingface-cli login`, and if you did pass a user token, double-check it's correct.

In [ ]:
# If using trainer API to train

from transformers import TrainingArguments

training_args = TrainingArguments(
    "bert-finetuned-mrpc",
    num_train_epochs = 3,
    save_strategy="epoch",
    push_to_hub=True,
    hub_model_id = "a_random_model"      # outside set as default outputdir ("bert-finetuned-mrpc")
)
# it will push each checkpoint (epoch) to hub


In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.1 MB/s eta 0:00:00


In [ ]:
# performing model load, dataset preprocess, all at once again
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from transformers import DataCollatorWithPadding, Trainer

checkpoint="bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

raw_datasets = load_dataset("glue","mrpc")      #MRPC (Microsoft Research Paraphrase Corpus) dataset used in GLUE benchmark

def tokenize_func(inp):
  return tokenizer(inp["sentence1"],inp["sentence2"],truncation = True)

tokenized_datasets = raw_datasets.map(tokenize_func, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,        # not mandatory to pass as it is the default only
    tokenizer=tokenizer,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

<ipython-input-14-73af1b201d12>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-67af56eb-5cb2cfd6662d2a9d6adf09a7;47a94ada-69b3-4e1d-a550-fdef3656b86c)

Invalid username or password.

In [ ]:
#trainer.train()

# pushes final model and meta-data (hyper-parameters and training results)
#trained.push_to_hub()

In [ ]:
# if u trained using pytorch like usual
# model.push_to_hub("sample_model_name")
# tokenizer.push_to_hub("sample_tok_name")

# for using a specific HF token
# model.push_to_hub("sample_model_name", use_auth_token="<TOKEN>")

#### push_to_hub() is a method of huggingface_hub package, which is installed with transformers package.

##### don't forget to login
```bash
huggingface-cli login
```

In [ ]:
# methods offered

from huggingface_hub import (
    # User management
    login,
    logout,
    whoami,

    # Repository creation and management
    create_repo,
    delete_repo,
    update_repo_visibility,

    # And some methods to retrieve/change information about the content
    list_models,
    list_datasets,
    list_repo_files,
    upload_file,
    delete_file,
)

In [ ]:
# from huggingface_hub import create_repo

# create_repo("dummy-model")

# arguments:
# private, in order to specify if the repository should be visible from others or not.
# token, if you would like to override the token stored in your cache by a given token.
# repo_type, if you would like to create a dataset or a space instead of a model. Accepted values are "dataset" and "space".

##### Version control is managed by **git** for smaller files and **git-lfs** for larger files.


#### Uploading files:

3 approaches:
- upload_file() API from huggingface_hub
- Repository class
- Git based approach

In [ ]:
# Upload file:  file size<5GB

# from huggingface_hub import upload_file

# upload_file(
#     "<path_to_file>/config.json",        # path of file to upload
#     path_in_repo="config.json",          # path of where to upload
#     repo_id="<namespace>/dummy-model",
# )

# repo_type = "dataset" or "space"           ## additional argument to upload on a non-model type repo

In [ ]:
# The Repository class

# git and git-lfs should be installed

# from huggingface_hub import Repository

# repo = Repository("<path_to_dummy_folder>", clone_from="<namespace>/dummy-model")         # clone and save repo


## Track large files using Git LFS (optional, but recommended for large models)
# repo.lfs_track(["*.bin", "*.h5", "*.ckpt"])  # Track model weight files

## <path_to_dummy_folder> : working directory and it has .gitaatributes file
# repo.git_pull()
# repo.git_add()
# repo.git_commit("message")
# repo.git_push()
# repo.git_tag()

GIT Based Approach (git and git-lfs should be installed)  -- my fav


To install git-lfs
```bash
git lfs install
```

Clone your repo:
```bash
git clone https://huggingface.co/<namespace>/<your-model-id>
```

Move to repo:
```bash
cd <your-model-id>
```

After saving model, tokenizer,.. in the repository.

Note: When creating the repository from the web interface, the *.gitattributes* file is automatically set up to consider files with certain extensions, such as *.bin* and *.h5*, as large files, and git-lfs will track them with no necessary setup on your side.

```bash
git add .
git status
git lfs track "*.bin" "*.h5"
git lfs status
git commit -m "First model version"
git push
```

Follow these commands


#### Model Card

It should preferably have these things:

- Model description
- Intended uses & limitations
- How to use
- Limitations and bias
- Training data
- Training procedure
- Evaluation results

